In [1]:
import functools
import itertools
import math
import re
import statistics
from functools import reduce
from pprint import pprint
from typing import (
    List,
    Optional,
    Generator,
    NamedTuple,
    Tuple,
    Set,
    Any,
    Dict,
    Union,
    Callable,
    Iterable,
    Sequence,
)

import rich.panel
import sklearn.base
from alive_progress import alive_bar
from numpy.distutils.misc_util import is_sequence
from tqdm import tqdm, trange
from log_symbols import LogSymbols

import overrides as overrides
import pandas as pds
import tabulate as tabulate
from collections import namedtuple
from sklearn import linear_model, metrics, preprocessing, model_selection, svm
from tabulate import tabulate
from pathlib import Path
import pickle
from halo import Halo
import random
import argparse
import sys
from sklearn.experimental import enable_halving_search_cv
from sklearn import neural_network, tree
from sklearn.model_selection import (
    HalvingGridSearchCV,
    RandomizedSearchCV,
    HalvingRandomSearchCV,
)
import scipy.stats as scistats

import rich.progress
import commons
from data_types import PickleOut
from rich.traceback import install as niceTracebacks
from rich.table import Table as rTable

from db_actions import db_actions
import numpy as np

from linRegPred import getSplits, getTransformers
from scipy.stats import rankdata
from sklearn.metrics import r2_score
import textdistance as TD

In [2]:
with db_actions.connect() as conn:
    dF = pds.read_sql('SELECT * FROM "runtimeScorePredBase1000"', conn)
# print(dF)
all_x_cols = [
    "build-linux-kernel1",
    "fio2",
    "fio3",
    "fio4",
    "fio5",
    "fio6",
    "fio7",
    "fio8",
    "fio9",
    "iperf10",
    "iperf11",
    "iperf12",
    "iperf13",
    "john-the-ripper14",
    "john-the-ripper15",
    "ramspeed16",
    "ramspeed17",
    "ramspeed18",
    "ramspeed19",
    "ramspeed20",
    "ramspeed21",
    "ramspeed22",
    "ramspeed23",
    "ramspeed24",
    "ramspeed25",
    "stream26",
    "stream27",
    "stream28",
    "stream29",
    "pCpu",
    "cpus",
    "rss",
    "vmem",
    "rchar",
    "wchar",
    "syscr",
    "syscw",
]
filtered_x_cols = ['john-the-ripper15', 'ramspeed17', 'ramspeed18', 'ramspeed19', 'ramspeed20', 'ramspeed21',
                   'ramspeed23', 'ramspeed24', 'ramspeed25', 'pCpu', 'cpus', 'vmem', 'rchar', 'wchar', 'syscr',
                   'syscw']
x_cols = all_x_cols
y_cols = "realtime"

X: pds.DataFrame = dF[x_cols]
y: pds.DataFrame = dF[y_cols]

In [3]:
from my_yaml import yaml_load

with open("nodeConfigIdLookup.yaml", "r") as f:
    t = yaml_load(f)
    nodeIDLUT = {v: k for k, v in t.items()}
dF["nodeName"] = dF["nodeConfig"].transform(lambda x: nodeIDLUT[x])

In [4]:
awsGroupLUT = {
    "general": ["M7g", "Mac", "M6g", "M6i", "M6in", "M6a", "M5", "M5n", "M5zn", "M5a", "M4", "A1", "T4g", "T3", "T3a",
                "T2"],
    "compute": ["C7g", "C7gn", "C6i", "C6in", "C6a", "C6g", "C6gn", "C5", "C5n", "C5a", "C4"],
    "memory": ["R7g", "R7iz", "R6g", "R6i", "R6in", "R6a", "R5", "R5n", "R5b", "R5a", "R4", "X2gd", "X2idn", "X2iedn",
               "X2iezn", "X1", "X1e", "z1d"],
    "accelerated": ["P4", "P3", "P2", "DL1", "Trn1", "Inf2", "Inf1", "G5", "G5g", "G4dn", "G4ad", "G3", "F1", "VT1"],
    "storage": ["Im4gn", "Is4gen", "I4i", "I3", "I3en", "D2", "D3", "D3en", "H1"],
    "HPC": ["Hpc6id", "Hpc6a"]
}

In [5]:
awsInstanceGroupLUT = {i.lower(): k for k, v in awsGroupLUT.items() for i in v}

In [6]:
dF["nodeGroup"] = dF["nodeName"].transform(lambda x: awsInstanceGroupLUT[x.split(".")[0]])

In [62]:
usedAwsGroupLUT = {
    k: list(pds.unique(dF.query("nodeGroup==@k").nodeName.values)) for k in awsGroupLUT.keys()
}

In [121]:
nodesPerGroup = {
    k: len(v) for k, v in usedAwsGroupLUT.items()
}

In [ ]:
for x in dF.groupby(by=["nodeGroup", "wfName", "taskName"]):
    c, d = x
    print(c, d.realtime.mean(), 100 * d.realtime.std() / d.realtime.mean(),
          100 * (d.realtime.max() - d.realtime.min()) / d.realtime.mean())

In [ ]:
import seaborn as sns

#sns.boxplot(data=dF,y="realtime",x="taskName",hue="nodeGroup")
for wf in dF.groupby(by="wfName"):
    wfname, data = wf
    print(wfname)
    grid = sns.FacetGrid(data, row="wfName", col="nodeGroup", sharex=False, sharey=True, margin_titles=True, height=5)
    grid.map_dataframe(sns.boxplot, y="realtime", x="taskName")

# Find prototypes for each group

#### failed attempt (i think?)

In [ ]:
#means = dF.groupby(by=["nodeGroup", "wfName", "taskName"]).realtime.mean().reset_index()
accu = dict()
for g in dF.groupby(by=["nodeGroup", "wfName", "taskName"]):
    l, d = g
    div = (d.realtime - d.realtime.mean()) ** 2 * d.realtime.mean()
    for i, nN in enumerate(d.nodeName.values):
        if nN not in accu.keys():
            accu[nN] = 0
        accu[nN] += div.values[i]



In [ ]:
print(accu)

#### good attempt

In [7]:
with db_actions.connect() as conn:
    scores_df = pds.read_sql('SELECT * FROM "nodeBenchmarkTransposedScores"', conn)


In [8]:
scores_df["nodeName"] = scores_df["nodeConfig"].transform(lambda x: nodeIDLUT[x])
scores_df["nodeGroup"] = scores_df["nodeName"].transform(lambda x: awsInstanceGroupLUT[x.split(".")[0]])

In [9]:
distances = dict()
benchmarkCols = [
    "build-linux-kernel1",
    "fio2",
    "fio3",
    "fio4",
    "fio5",
    "fio6",
    "fio7",
    "fio8",
    "fio9",
    "iperf10",
    "iperf11",
    "iperf12",
    "iperf13",
    "john-the-ripper14",
    "john-the-ripper15",
    "ramspeed16",
    "ramspeed17",
    "ramspeed18",
    "ramspeed19",
    "ramspeed20",
    "ramspeed21",
    "ramspeed22",
    "ramspeed23",
    "ramspeed24",
    "ramspeed25",
    "stream26",
    "stream27",
    "stream28",
    "stream29"]
for c in itertools.product(scores_df.query("nodeGroup=='general'").nodeName.values,
                           scores_df.query("nodeGroup=='compute'").nodeName.values,
                           scores_df.query("nodeGroup=='memory'").nodeName.values,
                           scores_df.query("nodeGroup=='storage'").nodeName.values):
    n1, n2, n3, n4 = c
    ns1 = scores_df.query("nodeName == @n1")[benchmarkCols].values[0]
    ns2 = scores_df.query("nodeName == @n2")[benchmarkCols].values[0]
    ns3 = scores_df.query("nodeName == @n3")[benchmarkCols].values[0]
    ns4 = scores_df.query("nodeName == @n4")[benchmarkCols].values[0]
    distances[c] = np.linalg.norm(ns1 - ns2) + np.linalg.norm(ns1 - ns3) + np.linalg.norm(ns1 - ns4) + np.linalg.norm(
        ns2 - ns3) + np.linalg.norm(ns2 - ns4) + np.linalg.norm(ns3 - ns4)

In [10]:
cs, ds = distances.keys(), distances.values()
#print("\n".join(map(lambda x: repr(x), sorted(zip(cs, ds), key=lambda x: x[1])[:10])))
groupPrototypes = sorted(zip(cs, ds), key=lambda x: x[1])[0][0]
print(groupPrototypes)

('m5.xlarge', 'c5.xlarge', 'r5.xlarge', 'i3.2xlarge')


In [14]:
bestGroupPrototypes = (
    'm5.xlarge', 'c5.xlarge', 'r5.xlarge', 'i3.2xlarge')  # = sorted(zip(cs, ds), key=lambda x: x[1])[0][0]
bestGroupPrototypesLUT = {
    awsInstanceGroupLUT[k.split(".")[0]]: k for k in bestGroupPrototypes
}

In [16]:
print(bestGroupPrototypesLUT)

{'general': 'm5.xlarge', 'compute': 'c5.xlarge', 'memory': 'r5.xlarge', 'storage': 'i3.2xlarge'}


# Produce Group Rankings

In [24]:
groupPerformances = dict()
for g in dF.groupby(by=["nodeGroup", "wfName", "taskName"]):
    l, d = g
    ng, wfn, tn = l
    if (wfn, tn) not in groupPerformances.keys():
        groupPerformances[(wfn, tn)] = dict()
    groupProto = bestGroupPrototypesLUT[ng]
    groupPerformances[(wfn, tn)][ng] = d.query("nodeName == @groupProto").realtime.mean()
    #groupPerformances[l] = d.realtime.mean()

In [19]:
groupOrder = ["general", "compute", "memory", "storage"]

In [20]:
groupRankings = pds.DataFrame(columns=["wfName", "taskName", *groupOrder])

In [21]:
for k, v in groupPerformances.items():
    wfn, tn = k
    performances = [v[i] for i in groupOrder]
    ranks = rankdata(performances, method="min")
    groupRankings = groupRankings.append({"wfName": wfn, "taskName": tn, **{o: r for o, r in zip(groupOrder, ranks)}},
                                         ignore_index=True)

In [22]:
print(groupRankings)

                   wfName                                           taskName  \
0    nfcore/chipseq:1.2.2                                          BWA_INDEX   
1    nfcore/chipseq:1.2.2                                            BWA_MEM   
2    nfcore/chipseq:1.2.2                                    CONSENSUS_PEAKS   
3    nfcore/chipseq:1.2.2                           CONSENSUS_PEAKS_ANNOTATE   
4    nfcore/chipseq:1.2.2                             CONSENSUS_PEAKS_DESEQ2   
..                    ...                                                ...   
74  nfcore/viralrecon:2.1  NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:NEXTC...   
75  nfcore/viralrecon:2.1  NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:PANGOLIN   
76  nfcore/viralrecon:2.1  NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:PLOT_...   
77  nfcore/viralrecon:2.1     NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:QUAST   
78  nfcore/viralrecon:2.1  NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:SNPEF...   

   general compute memory storage  
0  

#### Produce intragroup node rankings

In [25]:
nodeOrder = list(pds.unique(dF.nodeName))

In [76]:
groupNodePerformances = dict()
for g in dF.groupby(by=["nodeGroup", "wfName", "taskName", "nodeName"]):
    l, d = g
    ng, wfn, tn, nn = l
    l1 = (ng)
    l2 = (nn)
    if l1 not in groupNodePerformances.keys():
        groupNodePerformances[l1] = dict()
    if l2 not in groupNodePerformances[l1].keys():
        groupNodePerformances[l1][l2] = dict()
    groupNodePerformances[l1][l2][(wfn, tn)] = d.realtime.mean()
    #groupNodePerformances[(ng, wfn, tn)] = d.sort_values(by="nodeName", key=lambda v: sorted(list(v), key=lambda x: nodeOrder.index(x))).realtime.values

    #groupNodePerformances[l] = d.realtime.mean()

In [77]:
print(groupNodePerformances)

{'compute': {'c5.2xlarge': {('nfcore/chipseq:1.2.2', 'BWA_INDEX'): 6490.4, ('nfcore/chipseq:1.2.2', 'BWA_MEM'): 3933.0333333333333, ('nfcore/chipseq:1.2.2', 'CONSENSUS_PEAKS'): 1504.8, ('nfcore/chipseq:1.2.2', 'CONSENSUS_PEAKS_ANNOTATE'): 3566.2, ('nfcore/chipseq:1.2.2', 'CONSENSUS_PEAKS_DESEQ2'): 11615.8, ('nfcore/chipseq:1.2.2', 'FASTQC'): 6440.1, ('nfcore/chipseq:1.2.2', 'MACS2'): 2009.1, ('nfcore/chipseq:1.2.2', 'MACS2_ANNOTATE'): 3497.15, ('nfcore/chipseq:1.2.2', 'MACS2_QC'): 3674.0, ('nfcore/chipseq:1.2.2', 'MERGED_BAM'): 6674.033333333334, ('nfcore/chipseq:1.2.2', 'MERGED_BAM_FILTER'): 2290.133333333333, ('nfcore/chipseq:1.2.2', 'MERGED_BAM_REMOVE_ORPHAN'): 1211.3666666666666, ('nfcore/chipseq:1.2.2', 'MULTIQC'): 4601.0, ('nfcore/chipseq:1.2.2', 'PHANTOMPEAKQUALTOOLS'): 4162.566666666667, ('nfcore/chipseq:1.2.2', 'PICARD_METRICS'): 3847.2, ('nfcore/chipseq:1.2.2', 'PLOTFINGERPRINT'): 16625.0, ('nfcore/chipseq:1.2.2', 'PLOTPROFILE'): 88816.5, ('nfcore/chipseq:1.2.2', 'PRESEQ'): 2

In [153]:
tmp = pds.DataFrame(columns=["group", "nodeName", "wfName", "taskName", "realtime"])
for gr, v1 in groupNodePerformances.items():
    for nn, v2 in v1.items():
        for wfntn, r in v2.items():
            wfn, tn = wfntn
            tmp = tmp.append({"group": gr, "nodeName": nn, "wfName": wfn, "taskName": tn, "realtime": r},
                             ignore_index=True)

In [154]:
tmp2 = pds.DataFrame(
    columns=["group", "nodeName", "wfName", "taskName", "realtime", "groupRank", "intraGroupRank", "globalRank"])
for l, d in tmp.groupby(["group", "wfName", "taskName"]):
    gr, wfn, tn = l
    r = list(d.realtime.values)
    nn = list(d.nodeName.values)
    ra = list(rankdata(d.realtime, method="min"))
    cache = groupRankings.query("wfName==@wfn and taskName==@tn")
    ar = 0
    grr = cache[gr].values[0]
    for k, v in sorted(list(zip(groupOrder, (cache[k].values[0] for k in groupOrder))), key=lambda x: x[1]):
        if v >= grr:
            break
        ar += nodesPerGroup[k]
    for r1, nn1, ra1 in zip(r, nn, ra):
        tmp2 = tmp2.append(
            {"group": gr, "nodeName": nn1, "wfName": wfn, "taskName": tn, "realtime": r1,
             "intraGroupRank": ra1, "groupRank": grr, "globalRank": ar + ra1},
            ignore_index=True)

In [156]:
tmp3 = pds.DataFrame(
    columns=["group", "nodeName", "wfName", "taskName", "realtime", "groupRank", "intraGroupRank", "globalRank",
             "trueRank"])
for l, d in tmp2.groupby(["wfName", "taskName"]):
    wfn, tn = l
    r = list(d.realtime.values)
    nn = list(d.nodeName.values)
    ra = list(d.intraGroupRank.values)
    gr = list(d.group.values)
    grr = list(d.groupRank.values)
    grrr = list(d.globalRank.values)
    tr = list(rankdata(d.realtime, method="min"))
    for gr1, grr1, grrr1, r1, nn1, ra1, tr1 in zip(gr, grr, grrr, r, nn, ra, tr):
        tmp3 = tmp3.append(
            {"group": gr1, "nodeName": nn1, "wfName": wfn, "taskName": tn, "realtime": r1,
             "intraGroupRank": ra1, "groupRank": grr1, "globalRank": grrr1, "trueRank": tr1},
            ignore_index=True)

In [157]:
print(tmp3)

        group     nodeName                 wfName  \
0     compute   c5.2xlarge   nfcore/chipseq:1.2.2   
1     compute    c5.xlarge   nfcore/chipseq:1.2.2   
2     compute  c5a.2xlarge   nfcore/chipseq:1.2.2   
3     compute   c5a.xlarge   nfcore/chipseq:1.2.2   
4     general   m5.2xlarge   nfcore/chipseq:1.2.2   
...       ...          ...                    ...   
2128  storage   d3.2xlarge  nfcore/viralrecon:2.1   
2129  storage    d3.xlarge  nfcore/viralrecon:2.1   
2130  storage   i3.2xlarge  nfcore/viralrecon:2.1   
2131  storage     i3.large  nfcore/viralrecon:2.1   
2132  storage    i3.xlarge  nfcore/viralrecon:2.1   

                                               taskName     realtime  \
0                                             BWA_INDEX  6490.400000   
1                                             BWA_INDEX  9775.400000   
2                                             BWA_INDEX  6593.200000   
3                                             BWA_INDEX  5716.400000   
4  

In [160]:
print(np.linalg.norm(tmp3.trueRank - tmp3.globalRank))

310.56561303531333


In [166]:
export = pds.DataFrame({"wfName":tmp3.wfName, "taskName":tmp3.taskName, "nodeName": tmp3.nodeName,"rank":tmp3.globalRank})

In [167]:
export.to_csv("no_ML_ranks.csv",index=False)